In [ ]:
pip install python-amazon-sp-api

In [ ]:
from sp_api.base import Marketplaces, ReportType
from sp_api.api import Orders
from sp_api.api import Reports

creds = dict(
    refresh_token='x',
    lwa_app_id='x',
    lwa_client_secret='x',
    aws_access_key='x',
    aws_secret_key='x'
) 



In [ ]:
#inventory all listings report to get all ASINs under this account
from sp_api.api import Reports
import time

#code to get the key to download the report
report_types = "GET_MERCHANT_LISTINGS_ALL_DATA"
report = Reports(credentials=creds, marketplace=Marketplaces.US)
data = report.create_report(reportType=report_types)
report_id = data.payload['reportId'] 
print(data)
print('l\n')


#code to get the report
res = Reports(credentials=creds, marketplace=Marketplaces.US)
data = res.get_report(report_id)

while data.payload['processingStatus'] == "IN_QUEUE" or data.payload['processingStatus'] == 'IN_PROGRESS' :
  time.sleep(2)
  data = res.get_report(report_id)

if data.payload['processingStatus'] == "FATAL" or data.payload['processingStatus'] == "CANCELLED":
  print("Report failed!")
  report_data = data.payload
else:
  print("Success:")
  report_data = res.get_report_document(data.payload['reportDocumentId'])
  report_url = report_data.payload.get('url')
  print(report_data.payload)

In [ ]:
import requests
import csv


#code to make the data in a more disgesale manner/dictionary
url_request = requests.get(report_url)
decoded_content = url_request.content.decode('cp1252') #string type
reader = csv.DictReader(decoded_content.splitlines(), delimiter='\t') #csv.DictReader type


data_list = []
for row in reader:
  print(row)
  data = {
      'item-name': row['item-name'],
      'item_description': row['item-description'],
      'listing-id': row['listing-id'],
      'seller-sku': row['seller-sku'],
      'open-date': row['open-date'],
      'product-id': row['product-id'] 
      }
  data_list.append(data)

In [ ]:
from datetime import datetime

# Starting datetime
starting_datetime = datetime(2022,10,1)
starting_datetime_iso = starting_datetime.strftime('%Y-%m-%dT%H:%M:%S-05:00')
print('Starting Datetime:', starting_datetime)
print('Starting Datetime ISO: ', starting_datetime_iso)
print("\n")


# Ending datetime
ending_datetime = datetime(2023,7,1)
ending_datetime_iso = ending_datetime.strftime('%Y-%m-%dT%H:%M:%S-05:00')
print('Ending Datetime:', ending_datetime)
print('Ending Datetime ISO:', ending_datetime_iso)

In [ ]:
#the below code breaks down month by asin 

from sp_api.api.sales.sales import Granularity
from sp_api.api import Sales
from sp_api.base import Granularity
import pandas as pd


df0 = pd.DataFrame(columns=['ASIN',"Item Name", "Date", "Ordered Units", "Ordered Revenue"])


count = 0
for i in data_list:
  product_id = i['product-id']
  item_name = i['item-name']
  sales = Sales(credentials=creds, marketplace=Marketplaces.US)
  sales_request = sales.get_order_metrics(marketplaceIds="ATVPDKIKX0DER", interval=(f"{starting_datetime_iso}",f"{ending_datetime_iso}"),granularity=Granularity.MONTH, asin=product_id)
 
  z = 0
  for x in sales_request.payload:
    #print(product_id)
    #print(x['interval'])
    #print(sales_request.payload[z]['unitCount'])
    #print(sales_request.payload[z]['totalSales']["amount"])


    #first covnerting a date like 2022-10-01T00:00:00-05:00 into October 2022
    iso_date = sales_request.payload[z]['interval'].split('--')[0]
    iso_date_to_datetime =datetime.strptime(iso_date,'%Y-%m-%dT%H:%M-05:00')
    human_readable_date = iso_date_to_datetime.strftime('%B %Y') 

    #add data to the tabl
    data = {'ASIN': product_id,
            'Item Name': item_name,
            'Date':human_readable_date, 
            'Ordered Units':(sales_request.payload[z]['unitCount']), 
            'Ordered Revenue': (sales_request.payload[z]['totalSales']["amount"])}
    df0 = df0.append(data, ignore_index = True)

    z = z + 1
  count = count + 1

#printing how it looks
display(df0)

#code to reformat it the way customers want it
df0_reformatted = df0.pivot(index=['ASIN', "Item Name"], columns='Date')['Ordered Revenue']
display(df0_reformatted)

  


In [ ]:
from sp_api.api import Reports
import time

#code to get the key to download the rreport
report_types = "GET_SALES_AND_TRAFFIC_REPORT"
report = Reports(credentials=creds, marketplace=Marketplaces.US)
data = report.create_report(reportType=report_types, dataStartTime='2022-10-01T00:00:00.000Z', dataEndTime ="2022-12-31T00:00:00.000Z", reportOptions={"dateGranularity":"MONTH"})
report_id = data.payload['reportId'] #this doesn't work for some reason
print(data)
print(report_id)
print('l\n')


#code to get the report
res = Reports(credentials=creds, marketplace=Marketplaces.US)
data = res.get_report(report_id)

while data.payload['processingStatus'] == "IN_QUEUE" or data.payload['processingStatus'] == 'IN_PROGRESS' :
  time.sleep(2)
  data = res.get_report(report_id)

if data.payload['processingStatus'] == "FATAL" or data.payload['processingStatus'] == "CANCELLED":
  print("Report failed!")
  report_data = data.payload
else:
  print("Success:")
  report_data = res.get_report_document(data.payload['reportDocumentId'])
  report_url = report_data.payload.get('url')
  print(report_data.payload)


In [ ]:
from re import I
import requests
import csv
import json
from urllib.request import urlopen
import gzip



#code to make the data in a more disgesale manner/dictionary
url_request = requests.get(report_url)
decoded_content = gzip.decompress(url_request.content)
content = json.loads(decoded_content)


df2 = pd.DataFrame(columns=['Date', "PageViews", "Sessions", "Conversion Rate"])


x = 0
for i in content['salesAndTrafficByDate']:


  #first covnerting a date like 2022-10-01 into October 2022
  date = content['salesAndTrafficByDate'][x]['date']
  iso_date_to_datetime =datetime.strptime(date,'%Y-%m-%d')
  human_readable_date = iso_date_to_datetime.strftime('%B %Y') 

  data = {'Date':human_readable_date,
          'PageViews': (content['salesAndTrafficByDate'][x]['trafficByDate']['pageViews']),
          'Sessions': content['salesAndTrafficByDate'][x]['trafficByDate']['sessions'],
          'Conversion Rate': content['salesAndTrafficByDate'][x]['trafficByDate']['unitSessionPercentage']
          }
  df2 = df2.append(data, ignore_index = True)


  x = x + 1


In [ ]:
print(df0.dtypes) # somehow Ordered Units is an object
display(df0)


df0['Ordered Units'] = df0['Ordered Units'].astype(float)
df0 = df0.groupby("Date").mean()
display(df0)

In [ ]:
import pandas as pd
from IPython.display import display
pd.set_option('float_format', '{:,.1f}'.format)


#merging the two datasets
df3 = pd.merge(df0,df2,on='Date')
#df3.groupby('Date', axis=0)
#df3.groupby(pd.Grouper(key='Date', axis=0, freq='M'))



#adding MoM metrrics

df3['Ordered Revenue MoM'] = (df3['Ordered Revenue'].pct_change()*1)
df3['Ordered Revenue MoM'] = ["{:,.1%}".format(i) for i in df3['Ordered Revenue MoM']]


df3['Ordered Units MoM'] = (df3['Ordered Units'].pct_change()*1)
df3['Ordered Units MoM'] = ["{:,.1%}".format(i) for i in df3['Ordered Units MoM']]


df3['Sessions MoM'] = (df3['Sessions'].pct_change()*1)
df3['Sessions MoM'] = ["{:,.1%}".format(i) for i in df3['Sessions MoM']]



display(df0)
display(df2)
display(df3)


In [ ]:
#export our work
from google.colab import files

df0_reformatted.to_csv('Monthly_Revenue_By_ASIN.csv', encoding='utf-8')
files.download('Monthly_Revenue_By_ASIN.csv')

df3.to_csv(r'Monthly_KPIs_Performance.csv', encoding='utf-8')
files.download('Monthly_KPIs_Performance.csv')